In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import pandas as pd
import time

In [2]:
system_name = "Nugali"
location = "Brazil"
base_url = "https://pvoutput.org/list.jsp?p={}&id=81007&sid=71881&gs=0&v=0&o=date&d=desc"
pages_to_scrape = 5

In [3]:
options = Options()
options.binary_location = r"C:\Program Files\chrome for testing\chrome-win64\chrome.exe"
options.add_argument("--headless=new")
options.add_argument("--disable-gpu")
options.add_argument("--window-size=1920,1080")
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/134.0.0.0 Safari/537.36")
service = Service(executable_path=r"C:\Program Files\chrome for testing\chrome-win64\chromedriver.exe")
driver = webdriver.Chrome(service=service, options=options)

In [4]:
all_data = []

for page in range(pages_to_scrape):
    url = base_url.format(page)
    print(f"📄 Scraping page {page + 1} from: {url}")
    
    try:
        driver.get(url)
        time.sleep(6)
        soup = BeautifulSoup(driver.page_source, "html.parser")
        table = soup.find("table", id="tbl_main")

        if not table:
            print(f"⚠️ No table found on page {page + 1}. Retrying once...")
            time.sleep(5)
            driver.get(url)
            soup = BeautifulSoup(driver.page_source, "html.parser")
            table = soup.find("table", id="tbl_main")

        if table:
            rows = table.find_all("tr")[1:]
            for row in rows:
                cols = [td.get_text(strip=True).replace(',', '') for td in row.find_all("td")]
                if len(cols) >= 9:
                    cols.append(system_name)
                    cols.append(location)
                    all_data.append(cols)
        else:
            print(f"❌ Failed again: No table found on page {page + 1}")
    except Exception as e:
        print(f"🚨 Error scraping page {page + 1}: {e}")

driver.quit()

📄 Scraping page 1 from: https://pvoutput.org/list.jsp?p=0&id=81007&sid=71881&gs=0&v=0&o=date&d=desc
📄 Scraping page 2 from: https://pvoutput.org/list.jsp?p=1&id=81007&sid=71881&gs=0&v=0&o=date&d=desc
📄 Scraping page 3 from: https://pvoutput.org/list.jsp?p=2&id=81007&sid=71881&gs=0&v=0&o=date&d=desc
📄 Scraping page 4 from: https://pvoutput.org/list.jsp?p=3&id=81007&sid=71881&gs=0&v=0&o=date&d=desc
📄 Scraping page 5 from: https://pvoutput.org/list.jsp?p=4&id=81007&sid=71881&gs=0&v=0&o=date&d=desc


In [5]:
header = [
    "Date", "Generated", "Efficiency", "Exported", "Peak Power",
    "Peak Time", "Conditions", "Temperature", "Comments",
    "System Name", "Location"
]

In [6]:
df = pd.DataFrame(all_data, columns=header)
df.drop(columns=["Exported", "Temperature", "Comments", "Peak Power", "Peak Time", "Conditions"], inplace=True)
print(df.head())

       Date   Generated   Efficiency System Name Location
0  30/03/25  397.934kWh  3.685kWh/kW      Nugali   Brazil
1  29/03/25  440.045kWh  4.074kWh/kW      Nugali   Brazil
2  28/03/25  417.938kWh  3.870kWh/kW      Nugali   Brazil
3  27/03/25  410.776kWh  3.803kWh/kW      Nugali   Brazil
4  26/03/25  225.893kWh  2.092kWh/kW      Nugali   Brazil


In [7]:
page6 = pd.read_csv("Nugali_page6.csv")
page7 = pd.read_csv("Nugali_page7.csv")
page8 = pd.read_csv("Nugali_page8.csv")
page9 = pd.read_csv("Nugali_page9.csv")
page10 = pd.read_csv("Nugali_page10.csv")

In [8]:
page6["System Name"] = "Nugali"
page6["Location"] = "Brazil"
page7["System Name"] = "Nugali"
page7["Location"] = "Brazil"
page8["System Name"] = "Nugali"
page8["Location"] = "Brazil"
page9["System Name"] = "Nugali"
page9["Location"] = "Brazil"
page10["System Name"] = "Nugali"
page10["Location"] = "Brazil"

In [9]:
master_df = pd.concat([df, page6, page7, page8, page9, page10], ignore_index=True)

In [10]:
master_df

,Date,Generated,Efficiency,System Name,Location
0,30/03/25,397.934kWh,3.685kWh/kW,Nugali,Brazil
1,29/03/25,440.045kWh,4.074kWh/kW,Nugali,Brazil
2,28/03/25,417.938kWh,3.870kWh/kW,Nugali,Brazil
3,27/03/25,410.776kWh,3.803kWh/kW,Nugali,Brazil
4,26/03/25,225.893kWh,2.092kWh/kW,Nugali,Brazil
...,...,...,...,...,...
394,2/26/2024,253.094,2.343463,Nugali,Brazil
395,2/25/2024,134.42,1.24463,Nugali,Brazil
396,2/24/2024,476.083,4.408176,Nugali,Brazil
397,2/23/2024,572.466,5.300611,Nugali,Brazil


In [11]:
master_df["System Size (kW)"] = 108

In [12]:
master_df.to_csv("daily_Nugali_master.csv", index=False)